## **Importación de módulos y librerías**

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np
from numpy import array
from numpy import newaxis

#Data Split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

#Neuronal Network API
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras.regularizers import l2
from keras.models import load_model

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors

import os
import collections

In [ ]:
%matplotlib inline
print(plt.style.available)
plt.style.use('seaborn-v0_8')

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Lectura**

In [ ]:
df_columns = pd.read_csv('/content/drive/MyDrive/unet+lstm_regresion_one_step/imagenes_8x8_WT2.csv', nrows=1)

In [ ]:
variables = ['Amb_WindSpeed_Avg', 'Grd_Prod_Pwr_Avg', 'Blds_PitchAngle'] #,'Rtr_RPM_Avg']
variables_id = ['Id', 'Fecha_Inicio', 'Fecha_Fin', 'label']

columns_read = [col for col in df_columns.columns if any([v in col for v in variables + variables_id])]
df_wt2_completo = pd.read_csv('/content/drive/MyDrive/unet+lstm_regresion_one_step/imagenes_8x8_WT2.csv', usecols=columns_read)

In [ ]:
print(collections.Counter(df_wt2_completo['label']))

Counter({0: 39202, 1: 2934, -1: 818, 5: 424, 2: 227, 3: 135})


In [ ]:
df_wt2_completo['label'] = df_wt2_completo['label']+1
df_wt2_completo.loc[(df_wt2_completo['label']==6),'label'] = 5

In [ ]:
valores_distintos = df_wt2_completo['label'].value_counts()

# Imprimir el resultado
print("Número de valores distintos en la columna 'etiqueta_rendimiento':")
print(valores_distintos)

Número de valores distintos en la columna 'etiqueta_rendimiento':
label
1    39202
2     2934
0      818
5      424
3      227
4      135
Name: count, dtype: int64


In [ ]:
df_wt2_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43740 entries, 0 to 43739
Columns: 196 entries, Id to Blds_PitchAngle_Avg_64
dtypes: float64(192), int64(2), object(2)
memory usage: 65.4+ MB


In [ ]:
df_wt2_completo.head(1)

,Id,Fecha_Inicio,Fecha_Fin,label,Amb_WindSpeed_Avg_1,Amb_WindSpeed_Avg_2,Amb_WindSpeed_Avg_3,Amb_WindSpeed_Avg_4,Amb_WindSpeed_Avg_5,Amb_WindSpeed_Avg_6,...,Blds_PitchAngle_Avg_55,Blds_PitchAngle_Avg_56,Blds_PitchAngle_Avg_57,Blds_PitchAngle_Avg_58,Blds_PitchAngle_Avg_59,Blds_PitchAngle_Avg_60,Blds_PitchAngle_Avg_61,Blds_PitchAngle_Avg_62,Blds_PitchAngle_Avg_63,Blds_PitchAngle_Avg_64
0,2,2009-06-28 12:30:00,2009-06-28 23:10:00,5,4.7,4.6,5.3,5.5,3.8,5.8,...,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0


In [ ]:
df_wt2_completo.tail(1)

,Id,Fecha_Inicio,Fecha_Fin,label,Amb_WindSpeed_Avg_1,Amb_WindSpeed_Avg_2,Amb_WindSpeed_Avg_3,Amb_WindSpeed_Avg_4,Amb_WindSpeed_Avg_5,Amb_WindSpeed_Avg_6,...,Blds_PitchAngle_Avg_55,Blds_PitchAngle_Avg_56,Blds_PitchAngle_Avg_57,Blds_PitchAngle_Avg_58,Blds_PitchAngle_Avg_59,Blds_PitchAngle_Avg_60,Blds_PitchAngle_Avg_61,Blds_PitchAngle_Avg_62,Blds_PitchAngle_Avg_63,Blds_PitchAngle_Avg_64
43739,2,2017-03-09 21:40:00,2017-03-10 08:20:00,1,7.3,7.3,6.9,6.8,6.4,5.9,...,-1.0,-1.1,-1.1,-0.7,-0.1,0.1,0.0,-0.2,-0.6,-0.9


In [ ]:
# Número de etiquetas por año
años = pd.DataFrame()

años['Fecha_Inicio'] = pd.to_datetime(df_wt2_completo['Fecha_Inicio'])
años['Fecha_Fin'] = pd.to_datetime(df_wt2_completo['Fecha_Fin'])

# Extraemos el año de la columna 'inicio' y contamos las ocurrencias
conteo_por_año = años['Fecha_Inicio'].dt.year.value_counts().sort_index()

print(conteo_por_año)

Fecha_Inicio
2009    2785
2010    5688
2011    5702
2012    5717
2013    5714
2014    5688
2015    5663
2016    5709
2017    1074
Name: count, dtype: int64


## **Division en training, test,**
- El conjunto de entrenamiento recoge imagenes de 2009 - 2015
- El conjunto de prueba: 2016

In [ ]:
df_wt2_completo_copy = df_wt2_completo.copy()

In [ ]:
df_wt2_completo_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43740 entries, 0 to 43739
Columns: 196 entries, Id to Blds_PitchAngle_Avg_64
dtypes: float64(192), int64(2), object(2)
memory usage: 65.4+ MB


In [ ]:
# Convertimos las columnas de fecha en objetos datetime
df_wt2_completo_copy['Fecha_Inicio'] = pd.to_datetime(df_wt2_completo_copy['Fecha_Inicio'])
df_wt2_completo_copy['Fecha_Fin'] = pd.to_datetime(df_wt2_completo_copy['Fecha_Fin'])

# Filtramos las filas del año 2016
df_testing_2016 = df_wt2_completo_copy[df_wt2_completo_copy['Fecha_Inicio'].dt.year == 2016]

# Filtramos las filas excepto las del año 2016 ni las de 2017
df_training = df_wt2_completo_copy[(df_wt2_completo_copy['Fecha_Inicio'].dt.year != 2016) & (df_wt2_completo_copy['Fecha_Inicio'].dt.year != 2017)]

In [ ]:
# Obtenemos las etiquetas destino
performance_training = df_training["label"].to_numpy()
performance_testing = df_testing_2016["label"].to_numpy()

In [ ]:
print("Numero de etiquetas de entrenamiento: " + str(len(df_training)))
print("Numero de etiquetas de prueba: " + str(len(df_testing_2016)))

Numero de etiquetas de entrenamiento: 36957
Numero de etiquetas de prueba: 5709


In [ ]:
columnas_filtradas = [col for col in df_wt2_completo.columns if any(v in col for v in variables)]

dataset_training = df_training[columnas_filtradas].to_numpy()
dataset_testing = df_testing_2016[columnas_filtradas].to_numpy()

In [ ]:
dataset_training = np.swapaxes(dataset_training.reshape(-1,3,8,8),1,3)
dataset_testing = np.swapaxes(dataset_testing.reshape(-1,3,8,8),1,3)

In [ ]:
print(dataset_training.shape)
print(dataset_testing.shape)

(36957, 8, 8, 3)
(5709, 8, 8, 3)


In [ ]:
dataset_training =  tf.image.resize_with_pad(dataset_training, 16, 16, method='bicubic')
dataset_training = dataset_training.numpy().astype('float32')
print(dataset_training.shape)

dataset_testing =  tf.image.resize_with_pad(dataset_testing, 16, 16, method='bicubic')
dataset_testing = dataset_testing.numpy().astype('float32')
print(dataset_testing.shape)

(36957, 16, 16, 3)
(5709, 16, 16, 3)


In [ ]:
# split a univariate sequence into samples. Targets isnt normalized
def split_sequence(sequence, targets, n_steps_in, n_steps_out, future_steps, stride = None):
  if stride is  None:
    stride = 1
  X, y = list(), list()
  for i in range(0, len(sequence), stride):
    # find the end of this pattern
    end_ix = i + n_steps_in

    out_start_ix = end_ix + future_steps
    out_end_ix = out_start_ix + n_steps_out

    # check if we are beyond the sequence
    if out_end_ix > len(sequence):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], targets[out_start_ix:out_end_ix]
    X.append(seq_x)
    if n_steps_out == 1:
      y.append(seq_y[0])  # Si solo hay un paso de salida, añadir solo el valor
    else:
      y.append(seq_y)
  return array(X), array(y)

### **Preparacion training set**

In [ ]:
sequence_length = 65
output_steps = 1
batch_size = 1
future_steps = 5

In [ ]:
X_train, y_train = split_sequence(dataset_training, performance_training, sequence_length, output_steps, future_steps, stride = sequence_length)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(568, 65, 16, 16, 3)
(568,)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(batch_size)

In [ ]:
print(f"Número de batches train = secuencias: {sum(1 for _ in train_dataset)}")

Número de batches train = secuencias: 568


In [ ]:
i = 0
for samples, targets in train_dataset:
        print("it: ", i)
        print("samples shape:", samples.shape)
        print("targets shape:", targets.shape)
        print(targets.dtype)
        i = i + 1
        break

it:  0
samples shape: (1, 65, 16, 16, 3)
targets shape: (1,)
<dtype: 'int64'>


In [ ]:
def visualization (history):

  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='best')
  plt.show()

  # summarize history for MAE
  plt.plot(history.history['mae'])
  plt.plot(history.history['val_mae'])
  plt.title('Model MAE')
  plt.ylabel('MAE')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='best')
  plt.show()

### **Preparacion testing set**

In [ ]:
sequence_length = 65
output_steps = 1
batch_size = 1

In [ ]:
X_test, y_test = split_sequence(dataset_testing, performance_testing, sequence_length, output_steps, future_steps, stride = sequence_length)

In [ ]:
print(X_test.shape)
print(y_test.shape)

(87, 65, 16, 16, 3)
(87,)


In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)

In [ ]:
print(f"Número de batches test = secuencias: {sum(1 for _ in test_dataset)}")

Número de batches test = secuencias: 87


In [ ]:
i = 0
for samples, targets in test_dataset:
        print("it: ", i)
        print("samples shape:", samples.shape)
        print("targets shape:", targets.shape)
        print(targets.dtype)
        i = i + 1
        break

it:  0
samples shape: (1, 65, 16, 16, 3)
targets shape: (1,)
<dtype: 'int64'>


In [ ]:
def visualization (history):

  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='best')
  plt.show()

  # summarize history for MAE
  plt.plot(history.history['mae'])
  plt.plot(history.history['val_mae'])
  plt.title('Model MAE')
  plt.ylabel('MAE')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='best')
  plt.show()

## Funcion de prediccion

In [ ]:
### dataset: es la serie temporal de etiquetas
### output_steps: es el numero de steps futuros predichos por el modelo dada una Secuencia
### window_size es la longitud de cada secuencia
### n_sequences: indica el numero de secuencias a predecir. None para predecir todas las del dataset
### n_epochs: número de epochs el entrenamiento
def prediccion_secuencias_multiples(model, dataset, output_steps, window_size, n_sequences, n_epochs):

  prediction_seqs = []
  curr_frame = []
  stop_point = len(dataset) - window_size

  contador = 0

  # Iterar sobre el conjunto de datos con un paso de la longitud de secuencia
  for sequence, target in dataset:
      contador = contador + 1

      # Preparo la secuencia con la que se entrena el modelo para que contenga el estado interno lstm correspondiente a la siguiente secuencia que se desea predecir
      sample = tf.data.Dataset.from_tensor_slices((sequence, target))
      sample = sample.batch(1)

      prediction = model.predict(sample, verbose = 0)

      #print(f"### PREDICCION DEL MODELO: {prediction}")

      prediction = prediction.flatten()

      prediction_seqs.append(prediction)

      # Se entrena el modelo con la secuencia de prueba para que el modelo contenga el estado interno de la lsmt correspondiente para la siguiente prediccion de secuencia
      history = model.fit(sample, epochs = n_epochs, verbose = 0, shuffle = False)
      if(n_sequences != None and contador > n_sequences):
        break

  # Convertir la lista de predicciones en un array numpy
  prediction_seqs = np.array(prediction_seqs)

  return prediction_seqs

# **Modelos 1 EPOCH**

## ⭐ Modelo 0

In [ ]:
import tensorflow as tf
from keras.layers import Input, ConvLSTM2D, Flatten, RepeatVector, LSTM, TimeDistributed, Dense, BatchNormalization
from keras.models import Model

def build_model(input_layer=Input(batch_shape = (1, sequence_length, 16, 16, 3)), start_neurons=16):
    """Define la arquitectura ConvLSTM simple"""

    drop_rate_hidden = 0.4

    # Capa ConvLSTM2D
    conv_lstm = ConvLSTM2D(filters=32, kernel_size=(1,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(input_layer)
    conv_lstm = BatchNormalization()(conv_lstm)
    conv_lstm = ConvLSTM2D(filters=64, kernel_size=(1,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(conv_lstm)
    conv_lstm = BatchNormalization()(conv_lstm)
    conv_lstm = ConvLSTM2D(filters=64, kernel_size=(1,3), padding = "same", stateful = True, recurrent_dropout = 0.25)(conv_lstm)
    conv_lstm = BatchNormalization()(conv_lstm)
    flatten = Flatten()(conv_lstm)

    dense1 = Dense(64, activation='relu')(flatten)
    dense1 = Dropout(drop_rate_hidden)(dense1)
    outputs = Dense(1)(dense1)

    convlstm = Model(input_layer, outputs, name='ConvLSTM_simple_Regression')

    return convlstm

# Ejemplo de uso
convlstm = build_model()
convlstm.summary()

Model: "ConvLSTM_simple_Regression"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(1, 65, 16, 16, 3)]      0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (1, 65, 16, 16, 32)       13568     
                                                                 
 batch_normalization (Batch  (1, 65, 16, 16, 32)       128       
 Normalization)                                                  
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (1, 65, 16, 16, 64)       73984     
                                                                 
 batch_normalization_1 (Bat  (1, 65, 16, 16, 64)       256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (1, 16, 16,

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4)
convlstm.compile(optimizer="rmsprop", loss="mse", metrics=['mae'])

In [ ]:
history = convlstm.fit(train_dataset,
  epochs=1,
  validation_data=test_dataset,
  callbacks=[early_stopping], verbose=1, shuffle = False)

568/568 [==============================] - 300s 508ms/step - loss: 3.3775 - mae: 0.9938 - val_loss: 0.8468 - val_mae: 0.3220


In [ ]:
convlstm.save('/content/drive/MyDrive/TFG_modelado/ConvLSTM/Regresion/OneStep/Stateful/ConvLSTM simple/ConvLSTM_simple_24hras_1EPOCH_Model_0.h5')  # Guarda el modelo en tu Google Drive

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = convlstm.evaluate(test_dataset)
print("LOSS:", test_loss)
print("MAE:", test_mae)

87/87 [==============================] - 9s 102ms/step - loss: 0.8488 - mae: 0.3228
LOSS: 0.8487750291824341
MAE: 0.32275867462158203


In [ ]:
convlstm = load_model('/content/drive/MyDrive/TFG_modelado/ConvLSTM/Regresion/OneStep/Stateful/ConvLSTM simple/ConvLSTM_simple_24hras_1EPOCH.h5')

### Fase de prueba

#### **Predicción un valor sobre 2016 con stride = 1**

In [ ]:
sequence_length = 65
output_steps = 1
batch_size = 1
window_size = 65
n_sequences = 2000

In [ ]:
X_test_all, y_test_all = split_sequence(dataset_testing, performance_testing, sequence_length, output_steps, future_steps, stride = 1)

In [ ]:
print(X_test_all.shape)
print(y_test_all.shape)

(5639, 65, 16, 16, 3)
(5639,)


In [ ]:
test_dataset_all = tf.data.Dataset.from_tensor_slices((X_test_all, y_test_all))
test_dataset_all = test_dataset_all.batch(batch_size)

In [ ]:
print(f"Número de batches test = secuencias: {sum(1 for _ in test_dataset_all)}")

Número de batches test = secuencias: 5639


In [ ]:
predictions = prediccion_secuencias_multiples(convlstm, test_dataset_all, output_steps, window_size, n_sequences, 1)

In [ ]:
predictions.shape

(2001, 1)

In [ ]:
# Contar cuántas predicciones hay en cada rango
cantidad_0_1 = np.sum((predictions >= 0) & (predictions < 1))
cantidad_1_2 = np.sum((predictions >= 1) & (predictions < 2))
cantidad_2_3 = np.sum((predictions >= 2) & (predictions <= 3))
cantidad_3_4 = np.sum((predictions >= 3) & (predictions <= 4))
cantidad_4_5 = np.sum((predictions >= 4) & (predictions <= 5))

# Imprimir los resultados
print("Cantidad de predicciones entre 0 y 1:", cantidad_0_1)
print("Cantidad de predicciones entre 1 y 2:", cantidad_1_2)
print("Cantidad de predicciones entre 2 y 3:", cantidad_2_3)
print("Cantidad de predicciones entre 3 y 4:", cantidad_3_4)
print("Cantidad de predicciones entre 4 y 5:", cantidad_4_5)

Cantidad de predicciones entre 0 y 1: 1405
Cantidad de predicciones entre 1 y 2: 435
Cantidad de predicciones entre 2 y 3: 72
Cantidad de predicciones entre 3 y 4: 39
Cantidad de predicciones entre 4 y 5: 10


In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_secuencias_multiples_interactivo(prediction_seqs, window_size, dataset):
    # Calcular el promedio de las predicciones en cada punto de tiempo
    prediction_seqs = predictions.flatten()


    # Crear una lista para almacenar las trazas
    traces = []

    color_rojo_claro = 'hsl(0, 100%, 80%)'
    color_azul_claro = 'hsl(240, 100%, 80%)'

    # Graficar las predicciones promediadas
    x_values = list(range(sequence_length, len(dataset)))
    traces.append(go.Scatter(x=x_values, y=prediction_seqs, mode='lines', name='Predicciones promediadas', line=dict(color=color_azul_claro)))

    # Graficar los valores reales
    x_real_values = list(range(len(dataset)))
    traces.append(go.Scatter(x=x_real_values, y=dataset, mode='lines', name='Secuencia real', line=dict(color=color_rojo_claro)))

    # Crear el diseño del gráfico
    layout = go.Layout(
        title='Comparación de predicciones promediadas y secuencia real',
        xaxis=dict(title='Índice'),
        yaxis=dict(title='Valor'),
        showlegend=True
    )

    # Crear la figura
    fig = go.Figure(data=traces, layout=layout)

    # Mostrar el gráfico interactivo
    fig.show()

In [ ]:
performance_testing.shape

(5709,)

In [ ]:
plot_secuencias_multiples_interactivo(predictions, window_size, performance_testing)

## Modelo 1 Malo

In [ ]:
import tensorflow as tf
from keras.layers import Input, ConvLSTM2D, Flatten, RepeatVector, LSTM, TimeDistributed, Dense, Dropout
from keras.models import Model

def build_model(input_layer=Input(batch_shape = (1, sequence_length, 16, 16, 3)), start_neurons=16):
    """Define la arquitectura ConvLSTM simple"""

    drop_rate_hidden = 0.4

    # Capa ConvLSTM2D
    conv_lstm = ConvLSTM2D(filters=32, kernel_size=(1,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(input_layer)
    conv_lstm = ConvLSTM2D(filters=64, kernel_size=(1,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(conv_lstm)
    conv_lstm = ConvLSTM2D(filters=64, kernel_size=(1,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(conv_lstm)

    x = TimeDistributed(Flatten())(conv_lstm)
    # Capa LSTM
    lstm = LSTM(32, activation='relu', stateful = True, recurrent_dropout = 0.25, return_sequences = True)(x)
    lstm = LSTM(64, activation='relu', stateful = True, recurrent_dropout = 0.25)(lstm)
    dense1 = Dense(64,activation='relu')(lstm)
    dense1 = Dropout(drop_rate_hidden)(dense1)

    output_layer = Dense(1, name='Classifier')(dense1)
    convlstm = Model(input_layer, output_layer, name='ConvLSTM_simple_Regression')

    return convlstm

# Ejemplo de uso
convlstm = build_model()
convlstm.summary()

Model: "ConvLSTM_simple_Regression"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(1, 65, 16, 16, 3)]      0         
                                                                 
 conv_lstm2d_13 (ConvLSTM2D  (1, 65, 16, 16, 32)       13568     
 )                                                               
                                                                 
 conv_lstm2d_14 (ConvLSTM2D  (1, 65, 16, 16, 64)       73984     
 )                                                               
                                                                 
 conv_lstm2d_15 (ConvLSTM2D  (1, 65, 16, 16, 64)       98560     
 )                                                               
                                                                 
 time_distributed_2 (TimeDi  (1, 65, 16384)            0         
 stributed)                             

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4)
convlstm.compile(optimizer="rmsprop", loss="mse", metrics=['mae'])

In [ ]:
history = convlstm.fit(train_dataset,
  epochs=1,
  validation_data=test_dataset,
  callbacks=[early_stopping], verbose=1, shuffle = False)

568/568 [==============================] - 434s 748ms/step - loss: 14719506.0000 - mae: 565.2257 - val_loss: 50612381155328.0000 - val_mae: 1003651.7500


In [ ]:
convlstm.save('/content/drive/MyDrive/TFG_modelado/ConvLSTM/Regresion/OneStep/Stateful/ConvLSTM simple/ConvLSTM_simple_24hras_1EPOCH_Model_1.h5')  # Guarda el modelo en tu Google Drive

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = convlstm.evaluate(test_dataset)
print("LOSS:", test_loss)
print("MAE:", test_mae)

87/87 [==============================] - 13s 153ms/step - loss: 21829952.0000 - mae: 4092.6526
LOSS: 21829952.0
MAE: 4092.652587890625


In [ ]:
convlstm = load_model('/content/drive/MyDrive/TFG_modelado/ConvLSTM/Regresion/OneStep/Stateful/ConvLSTM simple/ConvLSTM_simple_24hras_1EPOCH_Model_1.h5')

### Fase de prueba

#### **Predicción un valor sobre 2016 con stride = 1**

In [ ]:
sequence_length = 65
output_steps = 1
batch_size = 1
window_size = 65
n_sequences = 2000

In [ ]:
X_test_all, y_test_all = split_sequence(dataset_testing, performance_testing, sequence_length, output_steps, future_steps, stride = 1)

In [ ]:
print(X_test_all.shape)
print(y_test_all.shape)

(5639, 65, 16, 16, 3)
(5639,)


In [ ]:
test_dataset_all = tf.data.Dataset.from_tensor_slices((X_test_all, y_test_all))
test_dataset_all = test_dataset_all.batch(batch_size)

In [ ]:
print(f"Número de batches test = secuencias: {sum(1 for _ in test_dataset_all)}")

Número de batches test = secuencias: 5639


In [ ]:
predictions = prediccion_secuencias_multiples(convlstm, test_dataset_all, output_steps, window_size, n_sequences, 1)

In [ ]:
predictions.shape

(2001, 1)

In [ ]:
# Contar cuántas predicciones hay en cada rango
cantidad_0_1 = np.sum((predictions >= 0) & (predictions < 1))
cantidad_1_2 = np.sum((predictions >= 1) & (predictions < 2))
cantidad_2_3 = np.sum((predictions >= 2) & (predictions <= 3))
cantidad_3_4 = np.sum((predictions >= 3) & (predictions <= 4))
cantidad_4_5 = np.sum((predictions >= 4) & (predictions <= 5))

# Imprimir los resultados
print("Cantidad de predicciones entre 0 y 1:", cantidad_0_1)
print("Cantidad de predicciones entre 1 y 2:", cantidad_1_2)
print("Cantidad de predicciones entre 2 y 3:", cantidad_2_3)
print("Cantidad de predicciones entre 3 y 4:", cantidad_3_4)
print("Cantidad de predicciones entre 4 y 5:", cantidad_4_5)

Cantidad de predicciones entre 0 y 1: 16
Cantidad de predicciones entre 1 y 2: 8
Cantidad de predicciones entre 2 y 3: 6
Cantidad de predicciones entre 3 y 4: 8
Cantidad de predicciones entre 4 y 5: 11


In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_secuencias_multiples_interactivo(prediction_seqs, window_size, dataset):
    # Calcular el promedio de las predicciones en cada punto de tiempo
    prediction_seqs = predictions.flatten()


    # Crear una lista para almacenar las trazas
    traces = []

    color_rojo_claro = 'hsl(0, 100%, 80%)'
    color_azul_claro = 'hsl(240, 100%, 80%)'

    # Graficar las predicciones promediadas
    x_values = list(range(sequence_length, len(dataset)))
    traces.append(go.Scatter(x=x_values, y=prediction_seqs, mode='lines', name='Predicciones promediadas', line=dict(color=color_azul_claro)))

    # Graficar los valores reales
    x_real_values = list(range(len(dataset)))
    traces.append(go.Scatter(x=x_real_values, y=dataset, mode='lines', name='Secuencia real', line=dict(color=color_rojo_claro)))

    # Crear el diseño del gráfico
    layout = go.Layout(
        title='Comparación de predicciones promediadas y secuencia real',
        xaxis=dict(title='Índice'),
        yaxis=dict(title='Valor'),
        showlegend=True
    )

    # Crear la figura
    fig = go.Figure(data=traces, layout=layout)

    # Mostrar el gráfico interactivo
    fig.show()

In [ ]:
performance_testing.shape

(5709,)

In [ ]:
plot_secuencias_multiples_interactivo(predictions, window_size, performance_testing)

## ⭐ Modelo 2 (es el modelo presentado en el TFG)
- cambia kernel a (3,3) con respecto Modelo 0

In [ ]:
import tensorflow as tf
from keras.layers import Input, ConvLSTM2D, Flatten, RepeatVector, LSTM, TimeDistributed, Dense, BatchNormalization
from keras.models import Model

def build_model(input_layer=Input(batch_shape = (1, sequence_length, 16, 16, 3)), start_neurons=16):
    """Define la arquitectura ConvLSTM simple"""

    drop_rate_hidden = 0.4

    # Capa ConvLSTM2D
    conv_lstm = ConvLSTM2D(filters=32, kernel_size=(3,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(input_layer)
    conv_lstm = BatchNormalization()(conv_lstm)
    conv_lstm = ConvLSTM2D(filters=64, kernel_size=(3,3), padding = "same", stateful = True, recurrent_dropout = 0.25, return_sequences = True)(conv_lstm)
    conv_lstm = BatchNormalization()(conv_lstm)
    conv_lstm = ConvLSTM2D(filters=64, kernel_size=(3,3), padding = "same", stateful = True, recurrent_dropout = 0.25)(conv_lstm)
    conv_lstm = BatchNormalization()(conv_lstm)
    flatten = Flatten()(conv_lstm)

    dense1 = Dense(64, activation='relu')(flatten)
    dense1 = Dropout(drop_rate_hidden)(dense1)
    outputs = Dense(1)(dense1)

    convlstm = Model(input_layer, outputs, name='ConvLSTM_simple_Regression')

    return convlstm

# Ejemplo de uso
convlstm = build_model()
convlstm.summary()

Model: "ConvLSTM_simple_Regression"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(1, 65, 16, 16, 3)]      0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (1, 65, 16, 16, 32)       40448     
                                                                 
 batch_normalization (Batch  (1, 65, 16, 16, 32)       128       
 Normalization)                                                  
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (1, 65, 16, 16, 64)       221440    
                                                                 
 batch_normalization_1 (Bat  (1, 65, 16, 16, 64)       256       
 chNormalization)                                                
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (1, 16, 16,

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=4)
convlstm.compile(optimizer="rmsprop", loss="mse", metrics=['mae'])

In [ ]:
history = convlstm.fit(train_dataset,
  epochs=1,
  validation_data=test_dataset,
  callbacks=[early_stopping], verbose=1, shuffle = False)

568/568 [==============================] - 319s 541ms/step - loss: 5.0146 - mae: 1.0553 - val_loss: 0.5580 - val_mae: 0.5096


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_mae = convlstm.evaluate(test_dataset)
print("LOSS:", test_loss)
print("MAE:", test_mae)

87/87 [==============================] - 11s 122ms/step - loss: 0.5510 - mae: 0.5078
LOSS: 0.5509611368179321
MAE: 0.5077868103981018


### Fase de prueba

#### **Predicción un valor sobre 2016 con stride = 1**

In [ ]:
sequence_length = 65
output_steps = 1
batch_size = 1
window_size = 65
n_sequences = 2000

In [ ]:
X_test_all, y_test_all = split_sequence(dataset_testing, performance_testing, sequence_length, output_steps, future_steps, stride = 1)

In [ ]:
print(X_test_all.shape)
print(y_test_all.shape)

(5639, 65, 16, 16, 3)
(5639,)


In [ ]:
test_dataset_all = tf.data.Dataset.from_tensor_slices((X_test_all, y_test_all))
test_dataset_all = test_dataset_all.batch(batch_size)

In [ ]:
print(f"Número de batches test = secuencias: {sum(1 for _ in test_dataset_all)}")

Número de batches test = secuencias: 5639


In [ ]:
predictions = prediccion_secuencias_multiples(convlstm, test_dataset_all, output_steps, window_size, n_sequences, 1)

In [ ]:
predictions.shape

(2001, 1)

In [ ]:
# Contar cuántas predicciones hay en cada rango
cantidad_0_1 = np.sum((predictions >= 0) & (predictions < 1))
cantidad_1_2 = np.sum((predictions >= 1) & (predictions < 2))
cantidad_2_3 = np.sum((predictions >= 2) & (predictions <= 3))
cantidad_3_4 = np.sum((predictions >= 3) & (predictions <= 4))
cantidad_4_5 = np.sum((predictions >= 4) & (predictions <= 5))

# Imprimir los resultados
print("Cantidad de predicciones entre 0 y 1:", cantidad_0_1)
print("Cantidad de predicciones entre 1 y 2:", cantidad_1_2)
print("Cantidad de predicciones entre 2 y 3:", cantidad_2_3)
print("Cantidad de predicciones entre 3 y 4:", cantidad_3_4)
print("Cantidad de predicciones entre 4 y 5:", cantidad_4_5)

Cantidad de predicciones entre 0 y 1: 1507
Cantidad de predicciones entre 1 y 2: 406
Cantidad de predicciones entre 2 y 3: 38
Cantidad de predicciones entre 3 y 4: 16
Cantidad de predicciones entre 4 y 5: 1


In [ ]:
import numpy as np
import plotly.graph_objects as go

def plot_secuencias_multiples_interactivo(prediction_seqs, window_size, dataset):
    # Calcular el promedio de las predicciones en cada punto de tiempo
    prediction_seqs = predictions.flatten()


    # Crear una lista para almacenar las trazas
    traces = []

    color_rojo_claro = 'hsl(0, 100%, 80%)'
    color_azul_claro = 'hsl(240, 100%, 80%)'

    # Graficar las predicciones promediadas
    x_values = list(range(sequence_length, len(dataset)))
    traces.append(go.Scatter(x=x_values, y=prediction_seqs, mode='lines', name='Predicciones promediadas', line=dict(color=color_azul_claro)))

    # Graficar los valores reales
    x_real_values = list(range(len(dataset)))
    traces.append(go.Scatter(x=x_real_values, y=dataset, mode='lines', name='Secuencia real', line=dict(color=color_rojo_claro)))

    # Crear el diseño del gráfico
    layout = go.Layout(
        title='Comparación de predicciones promediadas y secuencia real',
        xaxis=dict(title='Índice'),
        yaxis=dict(title='Valor'),
        showlegend=True
    )

    # Crear la figura
    fig = go.Figure(data=traces, layout=layout)

    # Mostrar el gráfico interactivo
    fig.show()

In [ ]:
performance_testing.shape

(5709,)

In [ ]:
plot_secuencias_multiples_interactivo(predictions, window_size, performance_testing)